In [1]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define a transform to normalize and resize images
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# Path where the data is stored (you can download manually from https://github.com/phelber/EuroSAT)
data_dir = 'Euro_SAT_RGB/EuroSAT_RGB'

# Load dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Create DataLoader
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Example: Check class names
print(dataset.classes)

['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']


In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class GeoCNN(nn.Module):
    def __init__(self):
        super(GeoCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)  # RGB bands (3 channels)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)

        self.fc1 = nn.Linear(128 * 8 * 8, 256)  # Adjust size after pooling
        self.fc2 = nn.Linear(256, 10)  # 10 land cover classes

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = x.view(-1, 128 * 8 * 8)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
# Initialize model, loss, optimizer
model = GeoCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [7]:
# Training loop
for epoch in range(10):
    running_loss = 0.0
    for images, labels in data_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(data_loader)}")

0
Epoch 1, Loss: 1.0787250647471414
1
Epoch 2, Loss: 0.6568726437846066
2
Epoch 3, Loss: 0.5588443713765856
3
Epoch 4, Loss: 0.4651489311220098
4
Epoch 5, Loss: 0.39127017593814584
5
Epoch 6, Loss: 0.33098343204954084
6
Epoch 7, Loss: 0.2746471749989413
7
Epoch 8, Loss: 0.23258770512355165
8
Epoch 9, Loss: 0.20169200589747932
9
Epoch 10, Loss: 0.15491414252090405


In [9]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in data_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total:.2f}%")

Accuracy: 95.30%
